# Download MODIS MCD12Q1_V6
Script based on example provided on: https://git.earthdata.nasa.gov/projects/LPDUR/repos/daac_data_download_python/browse

Requires a `.netrc` file in user's home directory with login credentials for `urs.earthdata.nasa.gov`. See: https://lpdaac.usgs.gov/resources/e-learning/how-access-lp-daac-data-command-line/

In [21]:
# modules
import os
import time
import urllib3
import requests
from netrc import netrc
from pathlib import Path
from shutil import copyfile
from datetime import datetime

#### Control file handling

In [2]:
# Easy access to control file folder
controlFolder = Path('../../../0_controlFiles')

In [3]:
# Store the name of the 'active' file in a variable
controlFile = 'control_active.txt'

In [4]:
# Function to extract a given setting from the control file
def read_from_control( file, setting ):
    
    # Open 'control_active.txt' and ...
    for line in open(file):
        
        # ... find the line with the requested setting
        if setting in line:
            break
    
    # Extract the setting's value
    substring = line.split('|',1)[1]      # Remove the setting's name (split into 2 based on '|', keep only 2nd part)
    substring = substring.split('#',1)[0] # Remove comments, does nothing if no '#' is found
    substring = substring.strip()         # Remove leading and trailing whitespace, tabs, newlines
    
    # Return this value    
    return substring

In [5]:
# Function to specify a default path
def make_default_path(suffix):
    
    # Get the root path
    rootPath = Path( read_from_control(controlFolder/controlFile,'root_path') )
    
    # Get the domain folder
    domainName = read_from_control(controlFolder/controlFile,'domain_name')
    domainFolder = 'domain_' + domainName
    
    # Specify the forcing path
    defaultPath = rootPath / domainFolder / suffix
    
    return defaultPath

#### Get the download settings

In [6]:
# Path and name of file with download links
links_path = read_from_control(controlFolder/controlFile,'parameter_land_list_path')
links_file = read_from_control(controlFolder/controlFile,'parameter_land_list_name')

In [7]:
# Specify the default paths if required 
if links_path == 'default':
    links_path = Path('./') # outputs a Path()
else:
    links_path = Path(links_path) # make sure a user-specified path is a Path()

In [8]:
# Find where the data needs to go
modis_path = read_from_control(controlFolder/controlFile,'parameter_land_raw_path')

In [9]:
# Specify the default paths if required 
if modis_path == 'default':
    modis_path = make_default_path('parameters/landclass/1_MODIS_raw_data') # outputs a Path()
else:
    modis_path = Path(modis_path) # make sure a user-specified path is a Path()

In [10]:
# Make output dir
modis_path.mkdir(parents=True, exist_ok=True)

#### Get the authentication info


In [11]:
# authentication url
url = 'urs.earthdata.nasa.gov'

In [12]:
# make the netrc directory
netrc_folder = os.path.expanduser("~/.netrc")

In [13]:
# Get user name and password - not great, but these are stored as plain text on the user's machine regardless..
usr = netrc(netrc_folder).authenticators(url)[0]
pwd = netrc(netrc_folder).authenticators(url)[2]

#### Do the downloads

In [14]:
# Get the download links from file
file_list = open(links_file, 'r').readlines()

In [26]:
# Count how many files we're trying to download
for i, l in enumerate(file_list):
    pass
num_files = i+1 # number of downloads

In [31]:
# The server connection is somewhat unstable, so keep trying until number of files in folder matches expectations
while len(os.listdir(modis_path)) < num_files:
    
    try:

        # Loop over the individual files and download
        for file_url in file_list:
    
            # Make the file name
            file_name = file_url.split('/')[-1].strip() # Get the last part of the url, strip whitespace and characters
    
            # Check if file already exists (i.e. interupted earlier download) and move to next file if so
            if (modis_path / file_name).is_file():
                continue 
    
            # Attempt a download
            with requests.get(file_url.strip(), verify=True, stream=True, auth=(usr,pwd)) as response:
    
                # Decode the response
                response.raw.decode_content = True
                content = response.raw
    
                # Save the response to file
                with open(modis_path / file_name, 'wb') as data:
                    while True:
                        chunk = content.read(16 * 1024)
                        if not chunk:
                            break
                        data.write(chunk)
                
            # Progress
            print('Successfully downloaded: {}'.format(file_name))
            time.sleep(3) # sleep for a bit so we don't overwhelm the server
            
    except: 
        print('--- Restarting download attempt ---')            

Successfully downloaded: MCD12Q1.A2007001.h21v02.006.2018145224009.hdf
Successfully downloaded: MCD12Q1.A2007001.h12v03.006.2018145220527.hdf
Successfully downloaded: MCD12Q1.A2007001.h21v11.006.2018145224104.hdf
Successfully downloaded: MCD12Q1.A2007001.h21v03.006.2018145224015.hdf
Successfully downloaded: MCD12Q1.A2007001.h11v10.006.2018145220434.hdf
Successfully downloaded: MCD12Q1.A2007001.h12v11.006.2018145221309.hdf
Successfully downloaded: MCD12Q1.A2007001.h28v14.006.2018145224801.hdf
Successfully downloaded: MCD12Q1.A2007001.h26v06.006.2018145224550.hdf
Successfully downloaded: MCD12Q1.A2007001.h18v17.006.2018145223232.hdf
Successfully downloaded: MCD12Q1.A2007001.h27v07.006.2018145224625.hdf
Successfully downloaded: MCD12Q1.A2007001.h10v04.006.2018145220023.hdf
Successfully downloaded: MCD12Q1.A2007001.h12v01.006.2018145220752.hdf
Successfully downloaded: MCD12Q1.A2007001.h15v01.006.2018145222149.hdf
Successfully downloaded: MCD12Q1.A2007001.h16v05.006.2018145222438.hdf
Succes

Successfully downloaded: MCD12Q1.A2007001.h11v08.006.2018145220414.hdf
Successfully downloaded: MCD12Q1.A2007001.h27v03.006.2018145224607.hdf
Successfully downloaded: MCD12Q1.A2007001.h14v11.006.2018145222050.hdf
Successfully downloaded: MCD12Q1.A2007001.h31v09.006.2018145225021.hdf
--- Restarting download attempt ---
Successfully downloaded: MCD12Q1.A2007001.h30v10.006.2018145224923.hdf
Successfully downloaded: MCD12Q1.A2007001.h30v06.006.2018145224903.hdf
Successfully downloaded: MCD12Q1.A2007001.h32v08.006.2018145225138.hdf
Successfully downloaded: MCD12Q1.A2007001.h30v13.006.2018145224943.hdf
Successfully downloaded: MCD12Q1.A2007001.h32v09.006.2018145225142.hdf
Successfully downloaded: MCD12Q1.A2007001.h33v08.006.2018145220705.hdf
Successfully downloaded: MCD12Q1.A2007001.h30v05.006.2018145224901.hdf
Successfully downloaded: MCD12Q1.A2007001.h33v11.006.2018145220905.hdf
Successfully downloaded: MCD12Q1.A2007001.h34v09.006.2018145221009.hdf
Successfully downloaded: MCD12Q1.A2007001

Successfully downloaded: MCD12Q1.A2008001.h12v13.006.2018145225823.hdf
Successfully downloaded: MCD12Q1.A2008001.h16v06.006.2018145230123.hdf
Successfully downloaded: MCD12Q1.A2008001.h17v16.006.2018145230313.hdf
Successfully downloaded: MCD12Q1.A2008001.h17v02.006.2018145230218.hdf
Successfully downloaded: MCD12Q1.A2008001.h02v10.006.2018145225121.hdf
Successfully downloaded: MCD12Q1.A2008001.h03v11.006.2018145225202.hdf
Successfully downloaded: MCD12Q1.A2008001.h07v06.006.2018145225303.hdf
Successfully downloaded: MCD12Q1.A2008001.h10v03.006.2018145225432.hdf
Successfully downloaded: MCD12Q1.A2008001.h19v08.006.2018145230512.hdf
Successfully downloaded: MCD12Q1.A2008001.h16v00.006.2018145230103.hdf
Successfully downloaded: MCD12Q1.A2008001.h09v08.006.2018145225418.hdf
Successfully downloaded: MCD12Q1.A2008001.h21v04.006.2018145230732.hdf
Successfully downloaded: MCD12Q1.A2008001.h10v04.006.2018145225442.hdf
Successfully downloaded: MCD12Q1.A2008001.h19v06.006.2018145230503.hdf
Succes

Successfully downloaded: MCD12Q1.A2008001.h13v12.006.2018145225913.hdf
Successfully downloaded: MCD12Q1.A2008001.h11v11.006.2018145225602.hdf
Successfully downloaded: MCD12Q1.A2008001.h18v14.006.2018145230413.hdf
Successfully downloaded: MCD12Q1.A2008001.h15v11.006.2018145230043.hdf
Successfully downloaded: MCD12Q1.A2008001.h20v17.006.2018145230712.hdf
Successfully downloaded: MCD12Q1.A2008001.h12v08.006.2018145225709.hdf
Successfully downloaded: MCD12Q1.A2008001.h02v08.006.2018145225111.hdf
Successfully downloaded: MCD12Q1.A2008001.h16v09.006.2018145230143.hdf
Successfully downloaded: MCD12Q1.A2008001.h08v07.006.2018145225333.hdf
Successfully downloaded: MCD12Q1.A2008001.h15v07.006.2018145230033.hdf
Successfully downloaded: MCD12Q1.A2008001.h20v12.006.2018145230652.hdf
Successfully downloaded: MCD12Q1.A2008001.h35v08.006.2018145225802.hdf
Successfully downloaded: MCD12Q1.A2008001.h22v14.006.2018145230942.hdf
Successfully downloaded: MCD12Q1.A2008001.h30v06.006.2018145231547.hdf
Succes

--- Restarting download attempt ---
Successfully downloaded: MCD12Q1.A2008001.h24v02.006.2018145231103.hdf
Successfully downloaded: MCD12Q1.A2008001.h31v06.006.2018145231807.hdf
Successfully downloaded: MCD12Q1.A2008001.h34v07.006.2018145225742.hdf
Successfully downloaded: MCD12Q1.A2008001.h23v08.006.2018145231032.hdf
Successfully downloaded: MCD12Q1.A2008001.h35v09.006.2018145225759.hdf
Successfully downloaded: MCD12Q1.A2005001.h27v14.006.2018145174824.hdf
Successfully downloaded: MCD12Q1.A2005001.h23v16.006.2018145172720.hdf
Successfully downloaded: MCD12Q1.A2005001.h25v02.006.2018145173223.hdf
Successfully downloaded: MCD12Q1.A2005001.h28v08.006.2018145175355.hdf
--- Restarting download attempt ---
Successfully downloaded: MCD12Q1.A2005001.h28v04.006.2018145174922.hdf
--- Restarting download attempt ---
Successfully downloaded: MCD12Q1.A2005001.h25v09.006.2018145173547.hdf
Successfully downloaded: MCD12Q1.A2005001.h27v04.006.2018145174152.hdf
--- Restarting download attempt ---
Succ

...

Successfully downloaded: MCD12Q1.A2013001.h19v06.006.2018146014028.hdf
Successfully downloaded: MCD12Q1.A2013001.h13v14.006.2018146013237.hdf
Successfully downloaded: MCD12Q1.A2013001.h17v07.006.2018146013728.hdf
Successfully downloaded: MCD12Q1.A2013001.h02v11.006.2018146012142.hdf
Successfully downloaded: MCD12Q1.A2013001.h15v15.006.2018146013431.hdf
Successfully downloaded: MCD12Q1.A2013001.h01v11.006.2018146012107.hdf
Successfully downloaded: MCD12Q1.A2013001.h11v03.006.2018146012903.hdf
Successfully downloaded: MCD12Q1.A2013001.h19v10.006.2018146014048.hdf
Successfully downloaded: MCD12Q1.A2013001.h17v03.006.2018146013710.hdf
Successfully downloaded: MCD12Q1.A2013001.h13v04.006.2018146013200.hdf
--- Restarting download attempt ---
Successfully downloaded: MCD12Q1.A2013001.h15v11.006.2018146013415.hdf
Successfully downloaded: MCD12Q1.A2013001.h00v08.006.2018146012022.hdf
Successfully downloaded: MCD12Q1.A2013001.h12v01.006.2018146013022.hdf
Successfully downloaded: MCD12Q1.A2013001

Successfully downloaded: MCD12Q1.A2015001.h23v16.006.2018146024549.hdf
Successfully downloaded: MCD12Q1.A2015001.h16v02.006.2018146023339.hdf
Successfully downloaded: MCD12Q1.A2015001.h13v03.006.2018146023039.hdf
Successfully downloaded: MCD12Q1.A2015001.h13v04.006.2018146023045.hdf
Successfully downloaded: MCD12Q1.A2015001.h23v03.006.2018146024455.hdf
Successfully downloaded: MCD12Q1.A2015001.h13v12.006.2018146023115.hdf
Successfully downloaded: MCD12Q1.A2015001.h14v17.006.2018146023224.hdf
Successfully downloaded: MCD12Q1.A2015001.h18v07.006.2018146023725.hdf
Successfully downloaded: MCD12Q1.A2015001.h22v07.006.2018146024354.hdf
Successfully downloaded: MCD12Q1.A2015001.h22v14.006.2018146024429.hdf
Successfully downloaded: MCD12Q1.A2015001.h23v09.006.2018146024524.hdf
Successfully downloaded: MCD12Q1.A2015001.h18v08.006.2018146023729.hdf
Successfully downloaded: MCD12Q1.A2015001.h10v04.006.2018146022724.hdf
--- Restarting download attempt ---
Successfully downloaded: MCD12Q1.A2015001

Successfully downloaded: MCD12Q1.A2015001.h30v08.006.2018146025144.hdf
--- Restarting download attempt ---
Successfully downloaded: MCD12Q1.A2015001.h29v09.006.2018146025054.hdf
Successfully downloaded: MCD12Q1.A2015001.h28v10.006.2018146024958.hdf
Successfully downloaded: MCD12Q1.A2015001.h32v09.006.2018146025319.hdf
Successfully downloaded: MCD12Q1.A2015001.h31v11.006.2018146025249.hdf
Successfully downloaded: MCD12Q1.A2015001.h30v09.006.2018146025150.hdf
Successfully downloaded: MCD12Q1.A2015001.h27v06.006.2018146024829.hdf
Successfully downloaded: MCD12Q1.A2015001.h26v03.006.2018146024734.hdf
Successfully downloaded: MCD12Q1.A2015001.h30v13.006.2018146025214.hdf
Successfully downloaded: MCD12Q1.A2015001.h34v09.006.2018146023814.hdf
Successfully downloaded: MCD12Q1.A2015001.h28v09.006.2018146024954.hdf
--- Restarting download attempt ---
Successfully downloaded: MCD12Q1.A2015001.h34v10.006.2018146023814.hdf
Successfully downloaded: MCD12Q1.A2015001.h30v06.006.2018146025134.hdf
Succe

Successfully downloaded: MCD12Q1.A2016001.h14v01.006.2018149125403.hdf
Successfully downloaded: MCD12Q1.A2016001.h16v12.006.2018149125650.hdf
Successfully downloaded: MCD12Q1.A2016001.h14v02.006.2018149125415.hdf
--- Restarting download attempt ---
Successfully downloaded: MCD12Q1.A2016001.h04v10.006.2018149124710.hdf
Successfully downloaded: MCD12Q1.A2016001.h08v05.006.2018149124823.hdf
Successfully downloaded: MCD12Q1.A2016001.h17v15.006.2018149125823.hdf
Successfully downloaded: MCD12Q1.A2016001.h10v03.006.2018149124956.hdf
Successfully downloaded: MCD12Q1.A2016001.h08v06.006.2018149124833.hdf
Successfully downloaded: MCD12Q1.A2016001.h03v10.006.2018149124655.hdf
Successfully downloaded: MCD12Q1.A2016001.h10v09.006.2018149125023.hdf
Successfully downloaded: MCD12Q1.A2016001.h16v00.006.2018149125603.hdf
Successfully downloaded: MCD12Q1.A2016001.h13v01.006.2018149125305.hdf
Successfully downloaded: MCD12Q1.A2016001.h12v05.006.2018149125214.hdf
Successfully downloaded: MCD12Q1.A2016001

Successfully downloaded: MCD12Q1.A2016001.h24v05.006.2018149130735.hdf
Successfully downloaded: MCD12Q1.A2016001.h30v05.006.2018149132529.hdf
Successfully downloaded: MCD12Q1.A2016001.h22v13.006.2018149130541.hdf
Successfully downloaded: MCD12Q1.A2016001.h20v12.006.2018149130243.hdf
Successfully downloaded: MCD12Q1.A2016001.h23v11.006.2018149130652.hdf
Successfully downloaded: MCD12Q1.A2016001.h31v09.006.2018149133044.hdf
Successfully downloaded: MCD12Q1.A2016001.h34v10.006.2018149133802.hdf
Successfully downloaded: MCD12Q1.A2016001.h32v09.006.2018149133502.hdf
Successfully downloaded: MCD12Q1.A2016001.h21v17.006.2018149130440.hdf
Successfully downloaded: MCD12Q1.A2016001.h27v14.006.2018149131817.hdf
Successfully downloaded: MCD12Q1.A2016001.h25v02.006.2018149130848.hdf
Successfully downloaded: MCD12Q1.A2016001.h32v07.006.2018149133328.hdf
Successfully downloaded: MCD12Q1.A2016001.h23v02.006.2018149130612.hdf
Successfully downloaded: MCD12Q1.A2016001.h34v08.006.2018149133703.hdf
Succes

#### Code provenance
Generates a basic log file in the domain folder and copies the control file and itself there.

In [32]:
# Set the log path and file name
logPath = modis_path
log_suffix = '_modis_download_log.txt'

In [33]:
# Create a log folder
logFolder = '_workflow_log'
Path( logPath / logFolder ).mkdir(parents=True, exist_ok=True)

In [34]:
# Copy this script
thisFile = 'download_modis_mcd12q1_v6.ipynb'
copyfile(thisFile, logPath / logFolder / thisFile);

In [35]:
# Get current date and time
now = datetime.now()

In [36]:
# Create a log file 
logFile = now.strftime('%Y%m%d') + log_suffix
with open( logPath / logFolder / logFile, 'w') as file:
    
    lines = ['Log generated by ' + thisFile + ' on ' + now.strftime('%Y/%m/%d %H:%M:%S') + '\n',
             'Downloaded MODIS MCD12Q1_V6 data with global coverage.']
    for txt in lines:
        file.write(txt) 